# **과제. 청주시 시설관리공단 민원 챗봇 만들기**

* 청주시의 시설관리공단에 있는 FAQ를 벡터DB로 만들어 RAG를 구성해 봅시다.
* https://www.cjsisul.or.kr/home/sub.do?menukey=48

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [ ]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

In [ ]:
path = '/content/drive/MyDrive/langchain/'

## **2.Vector DB 만들기**

* 시설관리 공단의 QA 데이터를 더 추가합니다.
    * 현재는 4건만 있음
* 추가한 csv 파일을 로딩해서 vectorDB를 구성
* 질문을 시도해 봅시다.
* 메모리를 구성해서

* 데이터 로딩

In [ ]:
csv_path = path + 'cj.csv'
data = pd.read_csv(csv_path, encoding='utf-8')
data.head()

* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : path + /db3



In [ ]:
db_path = path + '/db3'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory= db_path, embedding_function = embeddings )

* 데이터 입력
    * DF to Vector DB

In [ ]:
# 각 행의 데이터를 Document 객체로 변환
documents = [Document(page_content=text) for text in data.loc[0:1, 'QA'].tolist()]

In [ ]:
# 데이터프레임에서 문서 추가
database.add_documents(documents)

* 입력된 데이터 조회

In [ ]:
database.get()

In [ ]:
# 질문으로 유사도 검색해 보기
query = ""   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

In [ ]:
result[1][1]

## **3.RAG+LLM모델**

### (1) 기본모델 구성

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [ ]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

* 모델 선언

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

* 모델 확인

In [ ]:
query = ""
result = qa(query)

print(result["result"])

### (2) 메모리관리 모델 생성

In [ ]:
# 모델 선언


# Retriever 지정


# 대화 메모리 생성


# ConversationalRetrievalQA 체인 생성



* 모델 확인

In [ ]:
query = ""
result = qa(query)

print(result["result"])